In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import *
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold,train_test_split,GridSearchCV
from sklearn.metrics import log_loss

This Solution was prepared by SAI Krithik Rank 8 and Karan Juneja Rank 6, this code will run on kaggle and Colab

In [2]:
df_train = pd.read_csv("train.csv")

df_test = pd.read_csv("test.csv")


In [3]:
df_train['Hydro_fire'] = df_train['Horizontal_Distance_To_Fire_Points(meters)'] +  df_train['Horizontal_Distance_To_Hydrology(meters)']
df_train['Hydro_Road'] = df_train['Horizontal_Distance_To_Roadways(meters)'] +  df_train['Horizontal_Distance_To_Hydrology(meters)']
df_train['Road_fire'] = df_train['Horizontal_Distance_To_Fire_Points(meters)'] +  df_train['Horizontal_Distance_To_Roadways(meters)']

df_test['Hydro_fire'] = df_test['Horizontal_Distance_To_Fire_Points(meters)'] +  df_test['Horizontal_Distance_To_Hydrology(meters)']
df_test['Hydro_Road'] = df_test['Horizontal_Distance_To_Roadways(meters)'] +  df_test['Horizontal_Distance_To_Hydrology(meters)']
df_test['Road_fire'] = df_test['Horizontal_Distance_To_Fire_Points(meters)'] +  df_test['Horizontal_Distance_To_Roadways(meters)']

df_train['Hydro_Road_sub'] = np.abs(df_train['Horizontal_Distance_To_Roadways(meters)'] -  df_train['Horizontal_Distance_To_Hydrology(meters)'])
df_train['Hydro_fire_sub'] = np.abs(df_train['Horizontal_Distance_To_Fire_Points(meters)'] -  df_train['Horizontal_Distance_To_Hydrology(meters)'])
df_train['Road_fire_sub'] = np.abs(df_train['Horizontal_Distance_To_Fire_Points(meters)'] -  df_train['Horizontal_Distance_To_Roadways(meters)'])
df_test['Hydro_fire_sub'] = np.abs(df_test['Horizontal_Distance_To_Fire_Points(meters)'] -  df_test['Horizontal_Distance_To_Hydrology(meters)'])
df_test['Hydro_Road_sub'] = np.abs(df_test['Horizontal_Distance_To_Roadways(meters)'] -  df_test['Horizontal_Distance_To_Hydrology(meters)'])
df_test['Road_fire_sub'] = np.abs(df_test['Horizontal_Distance_To_Fire_Points(meters)'] -  df_test['Horizontal_Distance_To_Roadways(meters)'])

df_train['RAD_SLOPE'] = df_train['Slope(degrees)'].apply(lambda x: x*(np.pi/180))
df_train['RAD_Aspect'] = df_train['Aspect(degrees)'].apply(lambda x: x*(np.pi/180))
df_test['RAD_SLOPE'] = df_test['Slope(degrees)'].apply(lambda x: x*(np.pi/180))
df_test['RAD_Aspect'] = df_test['Aspect(degrees)'].apply(lambda x: x*(np.pi/180))


df_train['EL_DIS'] = df_train['Elevation(meters)'] - df_train['Horizontal_Distance_To_Hydrology(meters)']*0.2
df_train['EL_Fire'] = df_train['Elevation(meters)'] - df_train['Horizontal_Distance_To_Fire_Points(meters)']*0.2
df_train['EL_Road'] = df_train['Elevation(meters)'] - df_train['Horizontal_Distance_To_Roadways(meters)']*0.2
df_test['EL_DIS'] = df_test['Elevation(meters)'] - df_test['Horizontal_Distance_To_Hydrology(meters)']*0.2
df_test['EL_Fire'] = df_test['Elevation(meters)'] - df_test['Horizontal_Distance_To_Fire_Points(meters)']*0.2
df_test['EL_Road'] = df_test['Elevation(meters)'] - df_test['Horizontal_Distance_To_Roadways(meters)']*0.2

In [5]:
# df_train['Ele_hydro_add'] = df_train['Elevation(meters)'] + df_train['Vertical_Distance_To_Hydrology(meters)']
# df_train['Ele_hydro_sub'] = df_train['Elevation(meters)'] - df_train['Vertical_Distance_To_Hydrology(meters)']
# df_test['Ele_hydro_add'] = df_test['Elevation(meters)'] + df_test['Vertical_Distance_To_Hydrology(meters)']
# df_test['Ele_hydro_sub'] = df_test['Elevation(meters)'] - df_test['Vertical_Distance_To_Hydrology(meters)']

In [6]:
# df_train['Hill_9am_diff'] = df_train['Hillshade_Noon'] - df_train['Hillshade_9am']
# df_train['Hill_Noon_diff'] = df_train['Hillshade_Noon'] - df_train['Hillshade_3pm']
# df_train['Hillshade_9am_3pm']= df_train['Hillshade_9am'] - df_train['Hillshade_3pm']

# df_test['Hill_9am_diff'] = df_test['Hillshade_Noon'] - df_test['Hillshade_9am']
# df_test['Hill_Noon_diff'] = df_test['Hillshade_Noon'] - df_test['Hillshade_3pm']
# df_test['Hillshade_9am_3pm']= df_test['Hillshade_9am'] - df_test['Hillshade_3pm']

In [7]:
# df_train['Hyp_Elv_Hydro'] = np.sqrt((df_train['Horizontal_Distance_To_Hydrology(meters)'])**2 + (df_train['Elevation(meters)'])**2)
# df_train['Hyp_Elv_Fire'] = np.sqrt((df_train['Horizontal_Distance_To_Fire_Points(meters)'])**2 + (df_train['Elevation(meters)'])**2)
# df_train['Hyp_Elv_Road'] = np.sqrt((df_train['Horizontal_Distance_To_Roadways(meters)'])**2 + (df_train['Elevation(meters)'])**2)
# df_test['Hyp_Elv_Hydro'] = np.sqrt((df_test['Horizontal_Distance_To_Hydrology(meters)'])**2 + (df_test['Elevation(meters)'])**2)
# df_test['Hyp_Elv_Fire'] = np.sqrt((df_test['Horizontal_Distance_To_Fire_Points(meters)'])**2 + (df_test['Elevation(meters)'])**2)
# df_test['Hyp_Elv_Road'] = np.sqrt((df_test['Horizontal_Distance_To_Roadways(meters)'])**2 + (df_test['Elevation(meters)'])**2)

In [4]:
X_train = df_train.drop('Cover_Type',axis=1)
y_train = df_train['Cover_Type']

In [5]:
y_train.value_counts(normalize=True)

5    0.487608
7    0.364613
6    0.061549
1    0.035284
3    0.029880
2    0.016351
4    0.004716
Name: Cover_Type, dtype: float64

In [ ]:
splits = 5
folds = StratifiedKFold(n_splits=splits, shuffle=True, random_state=22)
# predictions = np.zeros((len(X_valid), 7))
oof_preds = np.zeros((len(df_test), 7))
feature_importance_df = pd.DataFrame()
final_preds = []
random_state = [77,89,22,1007,1997,1890,2000,2020,8989,786,787,1999992,2021,7654]
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train.values, y_train.values)):
        print("Fold {}".format(fold_))
        X_trn,y_trn = X_train.iloc[trn_idx],y_train.iloc[trn_idx]
        X_val,y_val = X_train.iloc[val_idx],y_train.iloc[val_idx]
        clf=LGBMClassifier(random_state=22,n_jobs=-1,max_depth=-1,min_data_in_leaf=17,num_leaves=67,colsample_bytree=0.9,
                        bagging_fraction=0.1,lambda_l2=1.1,n_estimators=5000,**params_gpu)
        clf.fit(X_trn, y_trn, eval_metric="logloss", eval_set=[(X_val,y_val)], verbose=True,early_stopping_rounds=100)
        final_preds.append(log_loss(y_pred=clf.predict_proba(X_val),y_true=y_val))
#         predictions += clf.predict_proba(X_valid)
        oof_preds += clf.predict_proba(df_test)

oof_preds = oof_preds/splits
print(sum(final_preds)/splits)

In [ ]:
print(log_loss(y_pred=oof_preds,y_true=y_test))

This with 10 folds and above features will give you a score of 0.25xx on lb , with xgboost you might get 0.23xx

Pseduo Labelling

In [ ]:
list2=[]
def checkconf(z):
    threshhold = 0.99
    if max(z)>threshhold :
        return np.argmax(z)+1
    else:
        return np.nan
for i in oof_preds:
    list2.append(checkconf(i))

the way we did pseudo labelling was using a stratification approach to keep the distribution same across the training dataset,
it worked amazingly with catboost but still need to refine it even more

In [ ]:
df_test['Cover_Type'] = list2
temp = df_test.dropna()

In [ ]:
df_train['Cover_Type'] = y_train

In [ ]:
temp_train = pd.concat([df_train,temp])
X_train = temp_train.drop('Cover_Type',axis=1)
y_train = temp_train['Cover_Type']

In [ ]:
catlist=[]
for i in range(len(X_train.columns)-1):
    if X_train.nunique()[i] <5:
        catlist.append(i)

To make the solution as reproducible as possible,i created a pickled file with new labels, as while using a GPU predictions change a bit so final predictions change too.

In [11]:
import pickle 
with open('psuedo_label_Train (1).pkl','rb') as f:
     pseudo_train_final = pickle.load(f)


In [13]:
pseudo_train_final.head()

,Elevation(meters),Aspect(degrees),Slope(degrees),Horizontal_Distance_To_Hydrology(meters),Vertical_Distance_To_Hydrology(meters),Horizontal_Distance_To_Roadways(meters),Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points(meters),...,Hydro_Road,Road_fire,Hydro_Road_sub,Hydro_fire_sub,Road_fire_sub,RAD_SLOPE,RAD_Aspect,EL_DIS,EL_Fire,EL_Road
80378,2893,27,17,42,-10,4377,211,202,123,3914,...,4419,8291,4335,3872,463,0.296706,0.471239,2884.6,2110.2,2017.6
344116,3367,259,21,201,57,3792,164,246,218,902,...,3993,4694,3591,701,2890,0.366519,4.520403,3326.8,3186.6,2608.6
407990,3323,170,12,531,103,1326,229,245,147,2136,...,1857,3462,795,1605,810,0.209440,2.967060,3216.8,2895.8,3057.8
303718,2317,318,26,30,12,860,144,207,198,849,...,890,1709,830,819,11,0.453786,5.550147,2311.0,2147.2,2145.0
556659,3357,104,23,180,8,2259,251,203,66,4542,...,2439,6801,2079,4362,2283,0.401426,1.815142,3321.0,2448.6,2905.2


In [ ]:
#Also Catboost gives different preditcions on different gpu  becasue of so nvidia gpu using single precision values so please take care of that
#Running the above pickled file in kaggle should reproduce a score of 
#0.2014-0.2017

In [ ]:
cb = CatBoostClassifier(random_state=27,task_type="GPU", devices="0:1",
                        cat_features = catlist,verbose=200,n_estimators=2800)

In [ ]:
sub = pd.DataFrame(preds,columns=[7,5,6,4,2,3,1])
sub = sub[[1,2,3,4,5,6,7]]
sub

In [ ]:
preds = cb.predict_proba(df_test,axis=1)
sub = pd.DataFrame(preds,columns=[1,2,3,4,5,6,7])
sub

Also getting 0.20xx with pseudo label is the only way of achieving that score, so if any other solution with out this is very difficult to achieve this score